# LLMFS Examples

This notebook demonstrates the usage of LLMFS (LLM-powered Memory Filesystem), a FUSE-based memory filesystem backed by JSON that can generate filesystem structures using OpenAI's GPT models.

## Setup

First, let's import the necessary components from LLMFS:

In [ ]:
import os
from dotenv import load_dotenv
from llmfs.llmfs import Memory, generate_filesystem

# Load environment variables from .env file
load_dotenv()

## Example 1: Structured Output with GPT-4

This example demonstrates how to use the GPT-4 model with structured outputs using Pydantic:

In [1]:
from pydantic import BaseModel
from openai import OpenAI
from typing import List

class ProjectStructure(BaseModel):
    name: str
    directories: List[str]
    files: List[str]
    description: str

client = OpenAI()
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract project structure information from the description."},
        {"role": "user", "content": "Create a web application project with a frontend directory for React components, a backend directory for API endpoints, and include necessary configuration files."},
    ],
    response_format=ProjectStructure,
)

project = completion.choices[0].message.parsed
print(f"Project Name: {project.name}")
print(f"Directories: {project.directories}")
print(f"Files: {project.files}")
print(f"Description: {project.description}")

# Verify structure
assert isinstance(project, ProjectStructure)
assert isinstance(project.name, str)
assert isinstance(project.directories, list)
assert isinstance(project.files, list)
assert isinstance(project.description, str)

# Verify content expectations
assert any("frontend" in d.lower() for d in project.directories), "Frontend directory not found"
assert any("backend" in d.lower() for d in project.directories), "Backend directory not found"
assert any(f.endswith(('.json', '.js', '.ts', '.yaml', '.yml')) for f in project.files), "No configuration files found"

Project Name: Web Application Project
Directories: ['frontend', 'backend']
Files: ['package.json', '.gitignore', 'README.md']
Description: This project is a web application consisting of a frontend developed with React and a backend for handling API endpoints. It includes essential configuration files typically required in a web development environment.


## Example 2: Generate a Python Project Structure

This example demonstrates how to use LLMFS to generate a Python project structure using natural language prompts. Note that this requires an OpenAI API key to be set in your environment.

In [ ]:
# Check if OpenAI API key is set
import os
from llmfs.llmfs import Memory
from llmfs.llmfs import generate_filesystem

if not os.getenv("OPENAI_API_KEY"):
    print("Warning: OPENAI_API_KEY not set. Skipping filesystem generation.")
else:
    # Generate filesystem structure
    prompt = "Create a Python project with src directory, tests, and documentation"
    fs_data = generate_filesystem(prompt)
    
    # Create filesystem instance
    fs = Memory(fs_data["data"])
    
    # Access generated structure
    root_children = fs._root._data["/"]["children"]
    print("Generated directories:", list(root_children.keys()))
    
    # Verify basic structure
    assert "src" in root_children
    assert "tests" in root_children
    assert "docs" in root_children
    print("✓ All expected directories present")

NameError: name 'generate_filesystem' is not defined

## Example 3: Manual Filesystem Operations

This example shows how to perform basic filesystem operations like creating directories, files, and symlinks.

In [ ]:
# Create filesystem instance
fs = Memory()

# Create a directory
fs.mkdir("/mydir", 0o755)
print("Created directory: /mydir")

# Create and write to a file
fd = fs.create("/mydir/hello.txt", 0o644)
fs.write("/mydir/hello.txt", b"Hello, World!", 0, fd)
print("Created and wrote to file: /mydir/hello.txt")

# Read file content
content = fs.read("/mydir/hello.txt", 1024, 0, fd)
print(f"File content: {content.decode('utf-8')}")
assert content.decode('utf-8') == "Hello, World!"

# Create a symlink
fs.symlink("/mydir/link", "hello.txt")
print("Created symlink: /mydir/link -> hello.txt")

# List directory contents
entries = fs.readdir("/mydir", None)
print(f"Directory contents: {sorted(entries)}")
assert sorted(entries) == ['.', '..', 'hello.txt', 'link']

## Example 4: Working with Extended Attributes

This example demonstrates how to work with extended attributes (xattr) in LLMFS.

In [ ]:
# Create filesystem instance
fs = Memory()

# Create a file
fd = fs.create("/metadata.txt", 0o644)
print("Created file: /metadata.txt")

# Set extended attributes
fs.setxattr("/metadata.txt", "user.author", "John Doe", None)
fs.setxattr("/metadata.txt", "user.version", "1.0", None)
print("Set extended attributes: author and version")

# Get extended attribute
author = fs.getxattr("/metadata.txt", "user.author")
print(f"Author attribute: {author}")
assert author == "John Doe"

# List extended attributes
xattrs = fs.listxattr("/metadata.txt")
print(f"All attributes: {sorted(xattrs)}")
assert sorted(xattrs) == ["user.author", "user.version"]

## Additional Examples

### Basic File Operations

This example shows additional file operations like truncating and deleting files.

In [ ]:
fs = Memory()

# Create and write to a file
fd = fs.create("/test.txt", 0o644)
fs.write("/test.txt", b"Initial content", 0, fd)
print("Created file with content: Initial content")

# Read content
content = fs.read("/test.txt", 1024, 0, fd)
print(f"Original content: {content.decode('utf-8')}")

# Truncate file
fs.truncate("/test.txt", 7)
content = fs.read("/test.txt", 1024, 0, fd)
print(f"Truncated content: {content.decode('utf-8')}")
assert content.decode('utf-8') == "Initial"

# Delete file
fs.unlink("/test.txt")
print("File deleted")
try:
    fs._root._data["/test.txt"]
    assert False, "File should not exist"
except KeyError:
    print("✓ File successfully deleted")

### Directory Operations

This example demonstrates nested directory operations.

In [ ]:
fs = Memory()

# Create nested directories
fs.mkdir("/parent", 0o755)
fs.mkdir("/parent/child", 0o755)
print("Created nested directories: /parent/child")

# List parent directory
entries = fs.readdir("/parent", None)
print(f"Parent directory contents: {sorted(entries)}")
assert sorted(entries) == ['.', '..', 'child']

# Remove empty directory
fs.rmdir("/parent/child")
entries = fs.readdir("/parent", None)
print(f"After removing child directory: {sorted(entries)}")
assert sorted(entries) == ['.', '..']